In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

## Load the Model

In [2]:
lstm_model = keras.models.load_model("lstm_model.h5")

## Helper Functions 

In [3]:
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.utils import pad_sequences

max_len = 35

def text_preprocessing(text):
    
    # Remove punctuations and numbers
    text = re.sub("[^a-zA-Z]"," ",text.lower())
    text = re.sub(r"\s+",' ',text)
    
    # Tokenization
    token = word_tokenize(text)
    
    # Remove StopWords
    token = [i for i in token if i not in stopwords.words("english")]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    token = [lemmatizer.lemmatize(word) for word in token]
    
    processed_text = " ".join(token)
    return processed_text

with open('tokenizer.pickle', 'rb') as handle:
    word_tokenizer = pickle.load(handle)
    
with open('encoder.pickle', 'rb') as handle:
    encoder = pickle.load(handle)

In [4]:
def string_to_vector(text):
    text = text_preprocessing(text)
    vector = word_tokenizer.texts_to_sequences([text])
    padded_vector = pad_sequences(vector,padding="post",maxlen=max_len)
    
    return padded_vector

## Predict Function

In [5]:
def predict(text):
    vector = string_to_vector(text)
    out = lstm_model.predict(vector,verbose=0)
    
    answer = np.argmax(out)
    ans = encoder.inverse_transform([answer])
    
    ans_dict = {}
    for i in range(5):
        ans_dict[encoder.inverse_transform([i])[0]] = (out[0][i]/np.sum(out) * 100)
        
    return ans[0],ans_dict

In [6]:
predict("I love movies")

('fear',
 {'anger': 24.688906967639923,
  'fear': 49.98914301395416,
  'joy': 16.8228879570961,
  'love': 3.4928038716316223,
  'sadness': 5.006255209445953})

In [7]:
predict("ghost movies are scary")

('fear',
 {'anger': 0.3404121147468686,
  'fear': 79.48245406150818,
  'joy': 3.675984963774681,
  'love': 0.31650534365326166,
  'sadness': 16.18463397026062})

In [8]:
predict("Ram lost a game")

('sadness',
 {'anger': 0.044325710041448474,
  'fear': 0.010724683670559898,
  'joy': 0.3180867759510875,
  'love': 0.020188411872368306,
  'sadness': 99.60667490959167})

In [9]:
predict("My mom scolded me for not eating my breakfast")

('anger',
 {'anger': 95.45241594314575,
  'fear': 0.4613462369889021,
  'joy': 2.2286556661129,
  'love': 0.6277773529291153,
  'sadness': 1.2298072688281536})

In [10]:
predict("she has a beautiful eyes and mesmerizing voice")

('love',
 {'anger': 0.664644269272685,
  'fear': 0.13790680095553398,
  'joy': 31.19540512561798,
  'love': 66.0341739654541,
  'sadness': 1.9678687676787376})